<a href="https://colab.research.google.com/github/ruymarinho013/CALNUM2025/blob/main/c_pia_de_sistemas_r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def gauss_jordan(A, b):
  #combina a matriz dos coeficientes A com os termos independentes b
  n = len(b)
  Ab = np.hstack([A.astype(float),b.reshape(-1,1).astype(float)])

  #Eliminação de Gauss-Jordan
  for i in range(n):
    #pivô principal
    pivo = Ab[i,i]
    if pivo ==0:
      raise ValueError("Pivô nulo! sistema pode não ter solução única")
    #normalização da linha do pivô
    Ab[i] = Ab[i]/pivo

    #zerar os outros elementos da coluna i
    for j in range(n):
      if j != i:
        fator = Ab[j, i]
        Ab[j] = Ab[j] - fator*Ab[i]
  x =Ab[:,-1]
  return x

In [ ]:
#Exemplo

A = np.array([[2, -1, 1],
              [3, 3, 9],
              [3, 3, 5]], dtype=float)
b = np.array([8, 0, -6], dtype=float)

solucao = gauss_jordan(A, b)
print("Solução do sistema:")
for i, val in enumerate(solucao, start=1):
  print(f"x{i} = {val:.4f}")

Solução do sistema:
x1 = 0.6667
x2 = -5.1667
x3 = 1.5000


##Atividade – Implementação do Método de Jacobi em Python

###Disciplina: Cálculo Numérico

###Enunciado

Você deverá implementar, em Python, o método de Jacobi para resolver sistemas lineares do tipo 𝐴𝑥=𝑏.

###Parte 1 – Função do método

Crie uma função em Python que:

receba a matriz 𝐴 , o vetor 𝑏 , um chute inicial, a tolerância e o número máximo de iterações;

calcule as iterações do método de Jacobi;

retorne:

a solução aproximada,o número de iterações realizadas,e mostre na tela, passo a passo, cada iteração (valores de 𝑥  e erro).

###Parte 2 – Teste do algoritmo

Na parte principal do programa (fora da função):

Defina pelo menos dois sistemas lineares diferentes (matrizes 𝐴  e vetores
𝑏 );

Para cada sistema, chame a função criada e exiba o resultado final.

###Requisitos

O código deve estar organizado com a função em um bloco e as chamadas da função em outro.

Em cada iteração, mostrar algo como:

Iteração 1: x = [ ... ]   erro = ...

Iteração 2: x = [ ... ]   erro = ...

ou uma tabela com os resultados de cada interação

###Objetivo da atividade

Compreender o funcionamento do método iterativo de Jacobi, implementando o algoritmo e analisando sua convergência através das aproximações sucessivas.

In [ ]:
import numpy as np
from tabulate import tabulate

def jacobi(A, b, x0=None, tol=1e-6, max_iter=100, verbose=True):
    """
    Resolve Ax = b usando o método iterativo de Jacobi.
    Exibe o progresso em formato de tabela.
    """
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    n = A.shape[0]

    if A.shape[0] != A.shape[1]:
        raise ValueError("A deve ser quadrada.")
    if b.size != n:
        raise ValueError("Dimensão de b incompatível com A.")
    if np.any(np.isclose(np.diag(A), 0.0)):
        raise ValueError("Há zeros na diagonal principal — método não aplicável sem pivotamento.")

    if x0 is None:
        x0 = np.zeros(n)
    else:
        x0 = np.array(x0, dtype=float)

    diag_dom = np.all(np.abs(np.diag(A)) > np.sum(np.abs(A), axis=1) - np.abs(np.diag(A)))
    if verbose:
        print("Iniciando Método de Jacobi")
        print(f"Tamanho do sistema: {n}x{n}")
        print(f"Tolerância: {tol}, Iterações máximas: {max_iter}")
        print("Observação:",
              "Matriz é estritamente diagonal dominante — boa chance de convergência.\n"
              if diag_dom else
              "Matriz NÃO é diagonalmente dominante — convergência não garantida.\n")

    D = np.diag(A)
    R = A - np.diagflat(D)
    x_old = x0.copy()
    historico = []

    for k in range(1, max_iter + 1):
        x_new = (b - R.dot(x_old)) / D
        err = np.max(np.abs(x_new - x_old))
        historico.append([k] + list(np.round(x_new, 6)) + [err])
        if err < tol:
            if verbose:
                print(tabulate(historico,
                               headers=["Iteração"] + [f"x{i+1}" for i in range(n)] + ["Erro"],
                               tablefmt="fancy_grid", floatfmt=".6f"))
                print(f"\n✅ Convergência alcançada em {k} iterações.")
                print(f"Solução aproximada: {np.round(x_new, 8).tolist()}")
            return x_new, k
        x_old = x_new

    if verbose:
        print(tabulate(historico,
                       headers=["Iteração"] + [f"x{i+1}" for i in range(n)] + ["Erro"],
                       tablefmt="fancy_grid", floatfmt=".6f"))
        print("\n⚠️  O método não convergiu dentro do número máximo de iterações.")
        print(f"Última aproximação: {np.round(x_old, 8).tolist()}")
    return x_old, max_iter


# ======== TESTES ==========
if __name__ == "__main__":
    # Sistema 1
    A1 = [[4.0, -1.0, 0.0],
          [-1.0, 4.0, -1.0],
          [0.0, -1.0, 4.0]]
    b1 = [15.0, 10.0, 10.0]
    x0_1 = [0.0, 0.0, 0.0]

    print("\n=== Sistema 1 ===")
    x_sol1, iters1 = jacobi(A1, b1, x0=x0_1, tol=1e-6, max_iter=100)

    # Sistema 2
    A2 = [[5.0, 2.0, 1.0],
          [1.0, 4.0, 1.0],
          [2.0, 1.0, 6.0]]
    b2 = [12.0, 7.0, 15.0]
    x0_2 = [0.0, 0.0, 0.0]

    print("\n=== Sistema 2 ===")
    x_sol2, iters2 = jacobi(A2, b2, x0=x0_2, tol=1e-6, max_iter=200)



=== Sistema 1 ===
Iniciando Método de Jacobi
Tamanho do sistema: 3x3
Tolerância: 1e-06, Iterações máximas: 100
Observação: Matriz é estritamente diagonal dominante — boa chance de convergência.

╒════════════╤══════════╤══════════╤══════════╤══════════╕
│   Iteração │       x1 │       x2 │       x3 │     Erro │
╞════════════╪══════════╪══════════╪══════════╪══════════╡
│          1 │ 3.750000 │ 2.500000 │ 2.500000 │ 3.750000 │
├────────────┼──────────┼──────────┼──────────┼──────────┤
│          2 │ 4.375000 │ 4.062500 │ 3.125000 │ 1.562500 │
├────────────┼──────────┼──────────┼──────────┼──────────┤
│          3 │ 4.765625 │ 4.375000 │ 3.515625 │ 0.390625 │
├────────────┼──────────┼──────────┼──────────┼──────────┤
│          4 │ 4.843750 │ 4.570312 │ 3.593750 │ 0.195312 │
├────────────┼──────────┼──────────┼──────────┼──────────┤
│          5 │ 4.892578 │ 4.609375 │ 3.642578 │ 0.048828 │
├────────────┼──────────┼──────────┼──────────┼──────────┤
│          6 │ 4.902344 │ 4.633789 │ 